In [ ]:
#the following shell commands demonstrate how to implement the variant calling pipeline on one sample
#assuming you have downloaded a sample: wgs_1011086.cram through Accesstoacram.ipynb

In [ ]:
# Step 1: extract reads from 48S regions that are known to the database/reference
# https://genome.ucsc.edu/cgi-bin/hgGateway
# chr21:8433222-8446571 chr21:8205988-8219301 chr21:8389035-8402342 chr22_KI270733v1_random:122273-135644 chrUn_GL000220v1:105424-118779
##########################################################################################################################################################
! samtools view -h wgs_1011086.cram chr21:8433222-8446571 chr21:8205988-8219301 chr21:8389035-8402342 chr22_KI270733v1_random:122273-135644 chrUn_GL000220v1:105424-118779 > RNA45SN1t5.cram

In [ ]:
# Step 2: reconstruct fastq from cram
####################################################################################################################################
%%bash
samtools collate -u -O RNA45SN1t5.cram | \
samtools fastq -1 paired1.fq -2 paired2.fq -0 /dev/null -s /dev/null -n


In [ ]:
# Step 3: alignment with Bowtie
# Bowtie2 installation: https://github.com/BenLangmead/bowtie2/releases/tag/v2.5.4
####################################################################################################################################
%%bash
/home/jupyter/packages/bowtie2-2.5.4-linux-x86_64/bowtie2 -5 1 -N 1 -p 8 \
    -x /home/jupyter/workspaces/humanrrnaproject/rDNA_prototype_prerRNA/rDNA_prototype_prerRNA_only \
    -1 paired1.fq \
    -2 paired2.fq \
    -S example_output.sam


In [ ]:
# Step 4: sam to bam; sorting
# the sorting here might be repetitive because you must sort it again after calling lofreq the first time,
# however this sort was necessary in the Greene, so recommand to keep
%%bash
samtools view -Sbh example_output.sam > example_output_rDNA.bam
samtools sort -@ 8 -o example_sort.bam -O 'bam' example_output_rDNA.bam


In [ ]:
# Step 5.1 : lofreq
# lofreq needs to be downloaded if you dont have it, do not recommand the installation method from lofreq website
# recommandation:
# 1.install miniconda: https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html
# 2. create a conda environment: conda create -n "lofreq"
# 3. install bioconda: conda config --add channels bioconda
# 4. install lofreq: conda install bioconda::lofreq
# 5. activate the environment with lofreq installed
%%bash
source ~/miniconda3/bin/activate
conda activate lofreq


In [ ]:
#Step 5.2 : mark indels
%%bash
lofreq indelqual --dindel \
    -f /home/jupyter/workspaces/humanrrnaproject/rDNA_prototype_prerRNA_only.fa \
    -o example_indel_rDNA.bam \
    example_output_rDNA.bam

In [ ]:
#Step 5.3 : SORT again, important to run before continuing
! samtools sort -@ 8 -o example_indel_sort.bam -O 'bam'  example_indel_rDNA.bam

In [ ]:
#Step 5.4 : generate vcf output
%%bash
lofreq call --call-indels -f /home/jupyter/workspaces/humanrrnaproject/rDNA_prototype_prerRNA_only.fa \
    -o example_output_rDNA.vcf \
    example_indel_sort.bam

In [ ]:
#Step 6 : coverage with bedtools
# bedtools has a pretty good bedtools.static.binary for installation, recommanded
# https://bedtools.readthedocs.io/en/latest/
! /home/jupyter/packages/bedtools genomecov -d -ibam example_output_rDNA.bam > example_coverage_test1.txt